## Bounds and constraints

In this exercise you learn how to use bounds and simple constraints. 

Note that we will just scratch the surface of the topic. Look at the resources for more information. 

## Resources

- [How to specify bounds](https://estimagic.readthedocs.io/en/stable/how_to_guides/optimization/how_to_specify_bounds.html)
- [How to use constraints](https://estimagic.readthedocs.io/en/stable/how_to_guides/optimization/how_to_specify_constraints.html)
- [Background: How constraints are implemented](https://estimagic.readthedocs.io/en/stable/explanations/optimization/implementation_of_constraints.html)

## Example

We reproduce the example from previous exercises for convenience.

In [1]:
import estimagic as em 
import numpy as np

def criterion(x):
    first = (x["a"] - np.pi) ** 2
    second =  np.sum((x["b"] - np.arange(3)) ** 2)
    third = np.sum((x["c"] - np.eye(2)) ** 2)
    return first + second + third
    
    
start_params = {
    "a": 1,
    "b": np.ones(3),
    "c": np.ones((2, 2))
}

In [2]:
res = em.minimize(
    criterion=criterion,
    params=start_params,
    algorithm="nlopt_bobyqa",
)

res.params

{'a': 3.141592653589793,
 'b': array([1.55630437e-16, 1.00000000e+00, 2.00000000e+00]),
 'c': array([[ 1.00000000e+00, -4.50891722e-18],
        [-4.51479888e-18,  1.00000000e+00]])}

## Task 1: Bounds

Repeat the optimization with an upper bounds of 2.0 on `a` and a lower bound of 0.5 for all entries in `b`.

## Solution 1:

In [3]:
res = em.minimize(
    criterion=criterion,
    params=start_params,
    algorithm="nlopt_bobyqa",
    lower_bounds={"b": 0.5 * np.ones_like(start_params["b"])},
    upper_bounds={"a": 2.0},
)

res.params

{'a': 2.0,
 'b': array([0.5, 1. , 2. ]),
 'c': array([[1., 0.],
        [0., 1.]])}

## Task 2: Fixing parameters

Remove the bounds but now fix the parameter `a` as well as the top right entry in `c` (i.e. `x["c"][0, 1]`) at their start value.

## Solution 2:

In [4]:
res = em.minimize(
    criterion=criterion,
    params=start_params,
    algorithm="nlopt_bobyqa",
    constraints=[
        {"type": "fixed", "selector": lambda x: x["a"]},
        {"type": "fixed", "selector": lambda x: x["c"][0, 1]},
    ],
)

res.params

{'a': 1.0,
 'b': array([0., 1., 2.]),
 'c': array([[1., 1.],
        [0., 1.]])}

## Optional: Play around with more constraints

Look at the [documentation](https://estimagic.readthedocs.io/en/stable/how_to_guides/optimization/how_to_specify_constraints.html) and impose the constraint that the parameters in `"c"` sum to 1.

## Solution:

In [5]:
# start params have to fulfill the constraint
start_params["c"] = np.tile(0.25, (2, 2))

res = em.minimize(
    criterion=criterion,
    params=start_params,
    algorithm="nlopt_bobyqa",
    constraints=[
        {"type":  "linear", "selector": lambda x: x["c"], "value": 1, "weights": 1.0}
    ],
)

res.params

/Users/Tim_Mensinger/miniconda3/envs/scipy-test/lib/python3.9/site-packages/pandas/core/frame.py:6254: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  diff = Index(subset).difference(self.columns)


{'a': 3.1415928282824384,
 'b': array([-7.47836576e-09,  9.99999785e-01,  2.00000000e+00]),
 'c': array([[ 0.75000093, -0.24999997],
        [-0.25000073,  0.74999977]])}

In [6]:
res.params["c"].sum()

1.0